In [1]:
import pandas as pd 
import os, glob
import datetime
import fiona
from shapely.geometry import Point, shape, Polygon
import geopandas as gpd
import shapely.speedups
import tqdm

In [2]:
countries_df = gpd.read_file("/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp")

In [3]:
countries_df=countries_df.rename(columns={'SOV_A3':'Country Code'})

In [4]:
countries_df.head()

,featurecla,scalerank,LABELRANK,SOVEREIGNT,Country Code,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,FCLASS_TR,FCLASS_ID,FCLASS_PL,FCLASS_GR,FCLASS_IT,FCLASS_NL,FCLASS_SE,FCLASS_BD,FCLASS_UA,geometry
0,Admin-0 country,0,2,Indonesia,IDN,0,2,Sovereign country,Indonesia,IDN,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.70361 4..."
1,Admin-0 country,0,3,Malaysia,MYS,0,2,Sovereign country,Malaysia,MYS,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((117.70361 4.16341, 117.69711 4..."
2,Admin-0 country,0,2,Chile,CHL,0,2,Sovereign country,Chile,CHL,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.50611..."
3,Admin-0 country,0,3,Bolivia,BOL,0,2,Sovereign country,Bolivia,BOL,...,None,None,None,None,None,None,None,None,None,"POLYGON ((-69.51009 -17.50659, -69.51009 -17.5..."
4,Admin-0 country,0,2,Peru,PER,0,2,Sovereign country,Peru,PER,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((-69.51009 -17.50659, -69.63832..."


In [5]:
df_earthquake=pd.read_csv('/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/emdat_date.csv',sep=',')

In [6]:
df_earthquake.head()

,Unnamed: 0,Dis No,Year,Month,Day,Disaster Type,Disaster Subtype,Country,Country Code,Region,...,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,myDt
0,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,NaN,2003000.0,NaN,NaN,NaN,NaN,550000.0,4813056.0,11.427251,1960-05-22
1,64,1960-0033-DZA,1960,2.0,21.0,Earthquake,Ground movement,Algeria,DZA,Northern Africa,...,1250.0,1250.0,NaN,NaN,NaN,NaN,NaN,NaN,11.427251,1960-02-21
2,65,1961-0016-ETH,1961,6.0,2.0,Earthquake,Ground movement,Ethiopia,ETH,Eastern Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.549601,1961-06-02
3,66,1962-0044-ALB,1962,3.0,18.0,Earthquake,Ground movement,Albania,ALB,Southern Europe,...,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,11.688060,1962-03-18
4,67,1962-0030-COL,1962,7.0,30.0,Earthquake,Ground movement,Colombia,COL,South America,...,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,11.688060,1962-07-30


In [7]:
df_countries_usgs=pd.read_csv('/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/merged_countries.csv',sep=',')

In [8]:
df_geo_usgs= gpd.GeoDataFrame(df_countries_usgs, geometry=gpd.points_from_xy(df_countries_usgs.longitude, df_countries_usgs.latitude))

In [9]:
df_geo_usgs.head()

,Unnamed: 0,geometry,souvereign,admin,index_right,time,latitude,longitude,depth,mag,id,place,Date
0,0,POINT (-74.46600 -8.49800),Peru,Peru,65172,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,13753,POINT (-74.46600 -8.49800),Peru,Peru,65172,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,0,POINT (-74.46600 -8.49800),Peru,Peru,1,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,13753,POINT (-74.46600 -8.49800),Peru,Peru,1,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
4,1,POINT (110.73600 35.51500),China,China,12,2010-01-24 00:00:00+00:00,35.515,110.736,28.7,5.0,usp000h6j0,"28 km ENE of Hancheng, China",2010-01-24


to match the emdat data set on country code - need to match the usgs data with country code

In [10]:
type(df_geo_usgs['geometry'])

geopandas.geoseries.GeoSeries

In [11]:
#getting geo fromat from usgs data into points 
points=df_geo_usgs['geometry']

In [12]:
points_list=[] #this list will contain geo points
country_code=[] #will contain country names related to the point
for p in tqdm.tqdm(points.values):
    containing_code = countries_df[countries_df['geometry'].contains(p)][['Country Code','ADM0_A3']].values
    
    if len(containing_code) > 0:
        points_list.append(p)
        country_code.append(containing_code)

100%|██████████| 17090/17090 [03:41<00:00, 77.27it/s] 


In [13]:
arr=country_code

v=[]#Country Code
j=[]#admin code
for i in arr:
    
    v.append(i.item(0))
    j.append(i.item(1))

In [14]:
d={'geometry':points_list,'Country Code':v, 'admin_code':j}
usgs_country_code=gpd.GeoDataFrame(d)

In [15]:
usgs_country_code

,geometry,Country Code,admin_code
0,POINT (-74.46600 -8.49800),PER,PER
1,POINT (-74.46600 -8.49800),PER,PER
2,POINT (-74.46600 -8.49800),PER,PER
3,POINT (-74.46600 -8.49800),PER,PER
4,POINT (110.73600 35.51500),CH1,CHN
...,...,...,...
17085,POINT (124.97800 40.16800),PRK,PRK
17086,POINT (156.67100 51.32400),RUS,RUS
17087,POINT (-100.98600 17.65500),MEX,MEX
17088,POINT (127.18700 -3.42800),IDN,IDN


In [16]:
type(usgs_country_code['geometry'])

geopandas.geoseries.GeoSeries

In [18]:
df_geo_usgs=df_geo_usgs.drop(columns=['index_right'])

In [19]:
df_country_codes = gpd.sjoin(usgs_country_code, df_geo_usgs, how="inner", op='intersects')

In [32]:
df_country_codes.head()

,geometry,Country Code,admin_code,index_right,Unnamed: 0,souvereign,admin,time,latitude,longitude,depth,mag,id,place,Date
0,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
0,POINT (-74.46600 -8.49800),PER,PER,2,0,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25


In [33]:
df_country_codes.reset_index(inplace=True)

In [34]:
df_country_codes.head()

,index,geometry,Country Code,admin_code,index_right,Unnamed: 0,souvereign,admin,time,latitude,longitude,depth,mag,id,place,Date
0,0,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,1,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,2,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,3,POINT (-74.46600 -8.49800),PER,PER,1,13753,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
4,0,POINT (-74.46600 -8.49800),PER,PER,2,0,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25


In [37]:
df_country_codes.drop(columns=['index','Unnamed: 0','index_right'],inplace=True)

In [38]:
df_country_codes.head()

,geometry,Country Code,admin_code,souvereign,admin,time,latitude,longitude,depth,mag,id,place,Date
0,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
1,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
2,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
3,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25
4,POINT (-74.46600 -8.49800),PER,PER,Peru,Peru,2010-01-25 00:00:00+00:00,-8.498,-74.466,146.7,5.9,usp000h6md,"13 km SE of San Fernando, Peru",2010-01-25


In [48]:
df_country_codes.to_csv(r'/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/usgs_countrycodes.csv')

In [ ]:
df_earthquake=df_earthquake.rename(columns={'myDt':'Date'})

In [113]:
#checking how many data points from the emdat earthquake file is in the usgs file
#1013 
df_earthquake[df_earthquake.Date.isin(df_country_codes.Date)]

,Unnamed: 0,Dis No,Year,Month,Day,Disaster Type,Disaster Subtype,Country,Country Code,Region,...,No Homeless,Total Affected,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damages ('000 US$),"Insured Damages, Adjusted ('000 US$)",Total Damages ('000 US$),"Total Damages, Adjusted ('000 US$)",CPI,Date
0,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,NaN,2003000.0,NaN,NaN,NaN,NaN,550000.0,4813056.0,11.427251,1960-05-22
1,64,1960-0033-DZA,1960,2.0,21.0,Earthquake,Ground movement,Algeria,DZA,Northern Africa,...,1250.0,1250.0,NaN,NaN,NaN,NaN,NaN,NaN,11.427251,1960-02-21
2,65,1961-0016-ETH,1961,6.0,2.0,Earthquake,Ground movement,Ethiopia,ETH,Eastern Africa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.549601,1961-06-02
3,66,1962-0044-ALB,1962,3.0,18.0,Earthquake,Ground movement,Albania,ALB,Southern Europe,...,NaN,154.0,NaN,NaN,NaN,NaN,NaN,NaN,11.688060,1962-03-18
4,67,1962-0030-COL,1962,7.0,30.0,Earthquake,Ground movement,Colombia,COL,South America,...,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,11.688060,1962-07-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1222,1541,1979-0300-TUR,1979,7.0,18.0,Earthquake,Ground movement,Turkey,TUR,Western Asia,...,1000.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,28.041681,1979-07-18
1225,1544,2019-0479-TUR,2019,9.0,26.0,Earthquake,Ground movement,Turkey,TUR,Western Asia,...,NaN,1453.0,NaN,NaN,NaN,NaN,NaN,NaN,98.781448,2019-09-26
1226,1545,2019-0306-USA,2019,7.0,5.0,Earthquake,Ground movement,United States of America (the),USA,Northern America,...,NaN,150.0,NaN,NaN,50000.0,50617.0,200000.0,202467.0,98.781448,2019-07-05
1228,1547,2020-0007-PRI,2020,1.0,7.0,Earthquake,Ground movement,Puerto Rico,PRI,Caribbean,...,900.0,5078.0,NaN,NaN,400000.0,400000.0,800000.0,800000.0,100.000000,2020-01-07


In [128]:
df_usgs_emdat=df_earthquake.merge(df_country_codes,how='inner', on=['Country Code','Date'])

In [129]:
df_usgs_emdat.head()

,Unnamed: 0,Dis No,Year,Month,Day,Disaster Type,Disaster Subtype,Country,Country Code,Region,...,admin_code,souvereign,admin,time,latitude,longitude,depth,mag,id,place
0,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,CHL,Chile,Chile,1960-05-22 00:00:00+00:00,-38.143,-73.407,25.0,9.5,official19600522191120_30,1960 Great Chilean Earthquake (Valdivia Earthq...
1,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,CHL,Chile,Chile,1960-05-22 00:00:00+00:00,-38.061,-73.039,25.0,7.8,official19600522185602_30,"38 km WNW of Traiguén, Chile"
2,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,CHL,Chile,Chile,1960-05-22 00:00:00+00:00,-37.860,-73.149,25.0,6.8,iscgem879128,"22 km ESE of Cañete, Chile"
3,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,CHL,Chile,Chile,1960-05-22 00:00:00+00:00,-37.775,-73.017,25.0,7.1,iscgem879127,"26 km W of Angol, Chile"
4,63,1960-0013-CHL,1960,5.0,22.0,Earthquake,Tsunami,Chile,CHL,South America,...,CHL,Chile,Chile,1960-05-22 00:00:00+00:00,-37.975,-73.127,25.0,5.8,iscgem879126,"30 km SE of Cañete, Chile"


In [130]:
df_usgs_emdat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1161 entries, 0 to 1160
Data columns (total 46 columns):
 #   Column                                     Non-Null Count  Dtype   
---  ------                                     --------------  -----   
 0   Unnamed: 0                                 1161 non-null   int64   
 1   Dis No                                     1161 non-null   object  
 2   Year                                       1161 non-null   int64   
 3   Month                                      1161 non-null   float64 
 4   Day                                        1161 non-null   float64 
 5   Disaster Type                              1161 non-null   object  
 6   Disaster Subtype                           1160 non-null   object  
 7   Country                                    1161 non-null   object  
 8   Country Code                               1161 non-null   object  
 9   Region                                     1161 non-null   object  
 10  Continent   

In [131]:
df_usgs_emdat.nunique()

Unnamed: 0                                    582
Dis No                                        582
Year                                           61
Month                                          12
Day                                            31
Disaster Type                                   1
Disaster Subtype                                2
Country                                        66
Country Code                                   66
Region                                         18
Continent                                       5
Location                                      570
Associated Dis                                  6
Associated Dis2                                 4
OFDA Response                                   1
Appeal                                          2
Declaration                                     2
Aid Contribution                               66
Dis Mag Value                                   7
Latitude                                      567


In [132]:
df_usgs_emdat.to_csv(r'/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/usgs_emdat_merged.csv')

In [170]:
test=pd.read_csv('/Users/tatiana/Documents/disaster_response/aws-disaster-response/data/test.csv',sep=',')

In [171]:
test.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

In [172]:
test[['time', 'latitude', 'longitude', 'depth', 'mag','place','id','magSource']]

,time,latitude,longitude,depth,mag,place,id,magSource
0,2005-12-24T02:01:55.020Z,35.231,136.840,43.0,4.7,"5 km ESE of Inazawa, Japan",usp000e6rd,nied
1,2005-12-21T07:09:05.170Z,-0.066,124.672,25.0,6.3,"153 km S of Tomohon, Indonesia",usp000e6ht,hrv
2,2005-12-18T04:23:10.010Z,2.842,95.916,18.2,5.7,"65 km NW of Sinabang, Indonesia",usp000e6bs,us
3,2005-12-16T18:32:16.420Z,38.506,141.900,42.9,6.0,"53 km E of Ishinomaki, Japan",usp000e69b,hrv
4,2005-12-14T07:09:54.190Z,30.476,79.255,44.0,5.1,"17 km WNW of P?palkoti, India",usp000e647,hrv
...,...,...,...,...,...,...,...,...
1520,2000-01-08T01:19:46.660Z,-9.805,159.811,33.0,6.4,"15 km SE of Malango, Solomon Islands",usp0009kvr,hrv
1521,2000-01-06T21:31:06.220Z,16.095,119.484,33.0,6.2,"Luzon, Philippines",usp0009kub,hrv
1522,2000-01-05T09:45:18.710Z,32.222,92.700,33.0,5.5,"102 km NE of Nagqu, China",usp0009ks9,hrv
1523,2000-01-03T22:34:12.640Z,22.132,92.771,33.0,4.6,"45 km SSW of Saiha, India",usp0009kqm,us


In [173]:
test.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2005-12-24T02:01:55.020Z,35.231,136.840,43.0,4.7,mwr,63.0,113.1,NaN,NaN,...,2020-07-08T02:00:09.919Z,"5 km ESE of Inazawa, Japan",earthquake,NaN,NaN,NaN,NaN,reviewed,jma,nied
1,2005-12-21T07:09:05.170Z,-0.066,124.672,25.0,6.3,mwc,380.0,40.3,NaN,0.99,...,2020-09-18T20:50:19.314Z,"153 km S of Tomohon, Indonesia",earthquake,NaN,NaN,NaN,NaN,reviewed,us,hrv
2,2005-12-18T04:23:10.010Z,2.842,95.916,18.2,5.7,mwb,133.0,40.1,NaN,1.16,...,2020-09-18T20:50:06.356Z,"65 km NW of Sinabang, Indonesia",earthquake,NaN,NaN,NaN,NaN,reviewed,us,us
3,2005-12-16T18:32:16.420Z,38.506,141.900,42.9,6.0,mwc,407.0,49.3,NaN,1.18,...,2020-09-18T20:49:56.071Z,"53 km E of Ishinomaki, Japan",earthquake,NaN,5.5,NaN,NaN,reviewed,us,hrv
4,2005-12-14T07:09:54.190Z,30.476,79.255,44.0,5.1,mwc,194.0,86.8,NaN,0.68,...,2020-07-08T01:59:55.833Z,"17 km WNW of P?palkoti, India",earthquake,NaN,8.2,NaN,NaN,reviewed,us,hrv


In [150]:
df_shakemap['rms']


0        NaN
1       0.99
2       1.16
3       1.18
4       0.68
        ... 
1535    1.30
1536    1.51
1537    0.82
1538    1.08
1539     NaN
Name: rms, Length: 1540, dtype: float64

In [174]:
df = gpd.read_file('mmi.geojson.json')

In [178]:
df[['id','mag','place','mmi']]

,id,mag,place,mmi
0,usp000e6rd,4.7,"5 km ESE of Inazawa, Japan",4.400
1,usp000e6ht,6.3,"153 km S of Tomohon, Indonesia",4.379
2,usp000e6bs,5.7,"65 km NW of Sinabang, Indonesia",5.604
3,usp000e69b,6.0,"53 km E of Ishinomaki, Japan",4.330
4,usp000e647,5.1,"17 km WNW of P?palkoti, India",4.943
...,...,...,...,...
1520,usp0009kvr,6.4,"15 km SE of Malango, Solomon Islands",6.226
1521,usp0009kub,6.2,"Luzon, Philippines",5.141
1522,usp0009ks9,5.5,"102 km NE of Nagqu, China",5.270
1523,usp0009kqm,4.6,"45 km SSW of Saiha, India",3.700
